In [15]:
"""
scrap all the links on the current page, 
extract anchor text into global variable -- "anchor_text"
return the url_set for the next depth
by filterring admin page, 
non-English page, avoiding outside link, 
ignoring navigations and marginal/side links; 
avoiding main Wikipedia page,disambiguation
"""
import os
from bs4 import BeautifulSoup 
from collections import defaultdict
import time
import re
import urllib.request
import urllib
from collections import deque
import requests 
import json
from itertools import product
from nltk.corpus import wordnet
from itertools import product
#import pyenchant


def get_current_pageurl_set(url,dom,en_keyword):
    #url_set=set() 
    url_set = deque()
    # ignore navigations and marginal/side links; ignore Non‐English articles
    inner_contents = dom.find("div", {"id":"mw-content-text","lang":"en"})
    #print(inner_contents)
    try:
        if len(inner_contents) > 0:
            #ignore external links
            external_links = inner_contents.findAll("a",{"class":"external"})
            for match in external_links:
                match.decompose()
            #links = inner_contents.findAll("a", href=True)
            #find_all <a href=/wiki..., avoid outside link
            links = inner_contents.findAll('a', attrs={'href': re.compile("^/wiki")})
            #linkre = re.compile('href=\"https://en.wikipedia.org/wiki(.+?)\"')
            #linkre.findall(inner_contents):
            #print(links)
            for link in links:
                wiki_url = link["href"].strip()
                archor_text = link.text
                # just Follow the links with the prefix /wiki, ignore # in current page,
                #linkre_wiki = re.compile("/wiki(.+?)").match(link_url).group(0)
                #print("linkre_wiki pattern match: " + wiki_url + "\r\n")
                #if linkre_wiki != None:
                #print("get inner url from /wiki/... pattern <--- " + wiki_url)
                #message = "get inner url from /wiki/... pattern <--- " + wiki_url
                #write_to_file('execute.log',message,False)
                ignore_filter = False
                # ignore administrative links, main Wikipedia page,disambiguation
                ignore_pattern_list = ["\D+:\D+","/wiki/Main_Page$","disambiguation"]
                for ignor in ignore_pattern_list:
                    try:
                        linkre_ignor = re.compile(ignor).search(wiki_url).group()
                        if linkre_ignor != None:
                            ignore_filter = True
                            filter_pattern = ignor
                            break
                    except AttributeError:
                        #print("Don't match pattern-- >"+ignor+ "\r\n")
                        message = "Don't match pattern-- >"+ignor+ "\r\n"
                        #write_to_file('execute.log',message,False)
                        continue
                if ignore_filter == True:
                    #print("Ignore the current url --->"+wiki_url + ". Pattern---> "+filter_pattern+ "\r\n")
                    f= open("ignore.log","a+",encoding="utf-8")
                    f.write("Ignore the current url:"+wiki_url+". Pattern: "+filter_pattern + "\r\n")
                    f.close()
                else:
                    if "#" in wiki_url:
                        wiki_url = wiki_url.rsplit('#', 1)[0]
                    url_set.append(wiki_url)
                    if en_keyword == True:
                        anchor_text.append(archor_text)
                    #print('Add URL to the queue --->  ' + wiki_url + "\r\n")
                    message = 'Add URL to the queue --->  ' + wiki_url + "\r\n"
                    #write_to_file('execute.log',message,False)
    except TypeError:
        message = url+" can't find the content block \r\n" + "pattern is: {id:mw-content-text,lang:en}\r\n" 
        log_name = "lostContentBlock.txt"
        #write_to_file(log_name,message,False)
         
    return url_set
    

# for G1

In [25]:
link_and_inlink = defaultdict(list)
flink_and_finlink = defaultdict(list)

path = os.getcwd() + "/doc1/" 
files = os.listdir(path) 

#map url to docId
url_docId = dict()
with open("docIdmap.txt", "r",encoding="utf-8",newline='\n') as infile1:
    for line in infile1:
        nodes = line.split(",")
        nodes[1] = nodes[1].rstrip()
        nodes[1] = nodes[1].replace("%","_")
        url_docId[nodes[1]] = nodes[0]
#print(url_docId)
#print('\n')
#get 1000 url list
furl = []
with open("G1Frontier.txt", "r",encoding="utf-8",newline='\n') as infile2:
    for line in infile2:
        nodes = line.split()
        #nodes[0] = nodes[0].replace(",","%2c")
        nodes[0] = nodes[0].replace("%","_")
        furl.append(nodes[0])
print(len(furl))

for url in furl:
    try:
        file_name = os.path.join(path,url_docId[url] + ".txt");
        print(file_name)
        with open(file_name, "r",encoding="utf-8",newline='\n') as infile:
            content = infile.read() 
            soup = BeautifulSoup(content, "html.parser")
            next_depth_url = get_current_pageurl_set(url,soup,False)
            for scrap_url in next_depth_url:
                fetch_url = "https://en.wikipedia.org" + scrap_url
                if fetch_url in furl:
                    docId = fetch_url.rsplit('/', 1)[-1]
                    inlink_docId = url.rsplit('/', 1)[-1]
                    link_and_inlink[docId].append(inlink_docId)
                    flink_and_finlink[fetch_url].append(url)               
    except KeyError:
        f = open("keyError.txt","a+",encoding="utf-8",newline='\n')
        f.write(url + ": the key is not in the dictionary" + "\n")
        f.close()
        #print(url + ": the key is not in the dictionary")
        continue
        #print(soup)

for childlink, parentlinks in link_and_inlink.items():
    parentlinks_uniqle = list(set(parentlinks))
    line = childlink +' '+ ' '.join(parentlinks_uniqle)+'\n'
    save_path = os.getcwd() + '/graph/'
    filename = os.path.join(save_path, "G1.txt")  
    f = open(filename,"a+",encoding="utf-8",newline='\n')
    f.write(line)
    f.close()
for childflink, parentflinks in flink_and_finlink.items():
    parentflinks_uniqle = list(set(parentflinks))
    save_path = os.getcwd() + '/graph/'
    filename = os.path.join(save_path, "FG1.txt") 
    line = childflink +' '+ ' '.join(parentflinks_uniqle)+'\n'
    f = open(filename,"a+",encoding="utf-8",newline='\n')
    f.write(line)
    f.close()                           
        
    
    




1000
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Carbon_footprint.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Carbon_dioxide_equivalent.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Carbon_dioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Methane.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Global_warming_potential.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Ecological_footprint.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Water_footprint.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Land_footprint.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Carbon_accounting.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Carbon_capture.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/EU.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/U.S..txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Luxembourg.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Australia.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Africa.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Ind

/Users/limingming/cs 6200/hw2/webGraphs/doc1/Sustainable_consumption.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Sustainability_and_systemic_change_resistance.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Tragedy_of_the_commons.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Population.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Birth_control.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Family_planning.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Human_population_control.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Zero_population_growth.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Appropriate_technology.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Environmental_technology.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Sustainable_design.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Biodiversity.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Biosecurity.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Biosphere.txt
/Users/limingming/cs 6200/hw2/

/Users/limingming/cs 6200/hw2/webGraphs/doc1/Radiative_forcing.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Tropospheric_ozone.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Natural_environment.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Albedo.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Bond_event.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Climate_oscillation.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Climate_sensitivity.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Cloud_forcing.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Cosmic_ray.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Climate_change_feedback.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Glacial_period.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Global_cooling.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Milankovitch_cycles.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Atlantic_multidecadal_oscillation.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/El_Ni%C3%B1o%E2%80%93

/Users/limingming/cs 6200/hw2/webGraphs/doc1/Climate_change_mitigation_scenarios.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Climate_engineering.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Reducing_emissions_from_deforestation_and_forest_degradation.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Reforestation.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Urban_reforestation.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Climate_change_adaptation.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Glacial_lake.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Desalination.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Drought_tolerance.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Irrigation.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Rainwater_tank.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Weather_modification.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Land_allocation_decision_support_system.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Glossary_o

/Users/limingming/cs 6200/hw2/webGraphs/doc1/Lake.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Ice_cap.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Glacier.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Seawater.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Petroleum.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Natural_gas.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Atmospheric_carbon_dioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Life.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Precambrian.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Photosynthesis.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Plant.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Algae.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Cyanobacteria.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Light.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Energy.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Carbohydrate.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Aerobic_

/Users/limingming/cs 6200/hw2/webGraphs/doc1/Yeast.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Ethanol.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Fatty_acid.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Anaerobic_respiration.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Monosaccharide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Anaerobic_organisms.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Chemical_kinetics.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Landfill_gas.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Urea.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Methanol.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Sodium_salicylate.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Kolbe-Schmitt_reaction.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Food_additive.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/E_number.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Leavening_agent.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Baker%27s_yea

/Users/limingming/cs 6200/hw2/webGraphs/doc1/Globally_Harmonized_System_of_Classification_and_Labelling_of_Chemicals.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/United_Nations_Economic_Commission_for_Europe.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/OECD_Guidelines_for_the_Testing_of_Chemicals.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Goma.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Mt._Nyiragongo.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Swahili_language.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Mazuku.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Apollo_13.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Carbon_dioxide_scrubber.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Respiratory_adaptation.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Acidosis.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Submarine.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/International_Space_Station.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/ASHRAE.

/Users/limingming/cs 6200/hw2/webGraphs/doc1/Magnesium_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Manganese(II)_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Mercury(II)_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Mercury_(element).txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Nickel(II)_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Nickel.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Nitric_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Palladium(II)_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Palladium.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Silicon_monoxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Strontium_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Strontium.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Sulfur_monoxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Sulfur.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Disulfur_dioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc

/Users/limingming/cs 6200/hw2/webGraphs/doc1/Rhenium_trioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Rhenium.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Selenium_trioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Sulfur_trioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Tellurium_trioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Tungsten_trioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Uranium_trioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Xenon_trioxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Xenon.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Iridium.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Dichlorine_heptoxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Manganese_heptoxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Rhenium(VII)_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Technetium(VII)_oxide.txt
/Users/limingming/cs 6200/hw2/webGraphs/doc1/Technetium.txt
/Users/limingming/cs 6200/hw2/

# for G2

In [14]:
link_and_inlink = defaultdict(list)
flink_and_finlink = defaultdict(list)

path = os.getcwd() + "/doc2/" 
files = os.listdir(path) 
#print(files)

#get 231 url list
furl = []
with open("G2Frontier.txt", "r",encoding="utf-8",newline='\n') as infile2:
    for line in infile2:
        nodes = line.split()
        nodes[0] = nodes[0].replace(",","%2c")
        furl.append(nodes[0])
furl = list(set(furl))
print(len(furl))

cnt = 0
for url in furl:
    cnt += 1 
    try:
        for file in files:
            file_name = os.path.join(path,file);
            #print(file_name)
            with open(file_name, "r",encoding="utf-8",newline='\n') as infile: 
                content = infile.read()
            if content.startswith(url):
                soup = BeautifulSoup(content, "html.parser")
                next_depth_url = get_current_pageurl_set(url,soup,False)
                for scrap_url in next_depth_url:
                    fetch_url = "https://en.wikipedia.org" + scrap_url
                    if fetch_url in furl:
                        docId = fetch_url.rsplit('/', 1)[-1]
                        inlink_docId = url.rsplit('/', 1)[-1]
                        link_and_inlink[docId].append(inlink_docId)
                        flink_and_finlink[fetch_url].append(url) 
                break
            else:
                continue
                
    except KeyError:
        f = open("keyError.txt","a+",encoding="utf-8",newline='\n')
        f.write(url + ": the key is not in the dictionary" + "\n")
        f.close()
        #print(url + ": the key is not in the dictionary")
        continue
        #print(soup)

for childlink, parentlinks in link_and_inlink.items():
    parentlinks_uniqle = list(set(parentlinks))
    line = childlink +' '+ ' '.join(parentlinks_uniqle)+'\n'
    save_path = os.getcwd() + '/graph/'
    filename = os.path.join(save_path, "G2.txt")  
    f = open(filename,"a+",encoding="utf-8",newline='\n')
    f.write(line)
    f.close()
for childflink, parentflinks in flink_and_finlink.items():
    parentflinks_uniqle = list(set(parentflinks))
    save_path = os.getcwd() + '/graph/'
    filename = os.path.join(save_path, "FG2.txt") 
    line = childflink +' '+ ' '.join(parentflinks_uniqle)+'\n'
    f = open(filename,"a+",encoding="utf-8",newline='\n')
    f.write(line)
    f.close()                           

    

['1.txt', '10.txt', '100.txt', '101.txt', '102.txt', '103.txt', '104.txt', '105.txt', '106.txt', '107.txt', '108.txt', '109.txt', '11.txt', '110.txt', '111.txt', '112.txt', '113.txt', '114.txt', '115.txt', '116.txt', '117.txt', '118.txt', '119.txt', '12.txt', '120.txt', '121.txt', '122.txt', '123.txt', '124.txt', '125.txt', '126.txt', '127.txt', '128.txt', '129.txt', '13.txt', '130.txt', '131.txt', '132.txt', '133.txt', '134.txt', '135.txt', '136.txt', '137.txt', '138.txt', '139.txt', '14.txt', '140.txt', '141.txt', '142.txt', '143.txt', '144.txt', '145.txt', '146.txt', '147.txt', '148.txt', '149.txt', '15.txt', '150.txt', '151.txt', '152.txt', '153.txt', '154.txt', '155.txt', '156.txt', '157.txt', '158.txt', '159.txt', '16.txt', '160.txt', '161.txt', '162.txt', '163.txt', '164.txt', '165.txt', '166.txt', '167.txt', '168.txt', '169.txt', '17.txt', '170.txt', '171.txt', '172.txt', '173.txt', '174.txt', '175.txt', '176.txt', '177.txt', '178.txt', '179.txt', '18.txt', '180.txt', '181.txt'

/Users/limingming/cs 6200/hw2/webGraphs
